# Multithreading e Multiprocessamento

Lembre-se da frase "muitas mãos facilitam o trabalho". Isso é tão verdadeiro na programação quanto em qualquer outro lugar.

E se você pudesse projetar seu programa Python para fazer quatro coisas ao mesmo tempo? O que normalmente levaria uma hora poderia (quase) levar um quarto do tempo. 

Essa é a idéia por trás do processamento paralelo ou a capacidade de configurar e executar várias tarefas simultaneamente.


## Multiprocessamento Exemplo: Monte Carlo

Vamos codificar um exemplo para ver como as peças se encaixam. Podemos cronometrar nossos resultados usando o módulo **timeit** para medir quaisquer ganhos de desempenho. Nossa tarefa é aplicar o método de Monte Carlo para estimar o valor de Pi.

In [2]:
from random import random  # execute essa importação fora da função

def find_pi(n):
    """
    Função para estimar o valor de Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # se i cai dentro do círculo
            inside+=1

    pi=4*inside/n
    return pi

Vamos testar o `find_pi` em 5.000 pontos:

In [3]:
find_pi(5000)

3.128

In [4]:
%%writefile test.py
from random import random
from multiprocessing import Pool
import timeit

def find_pi(n):
    """
    Função para estimar o valor de Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # se i cai dentro do círculo
            inside+=1

    pi=4*inside/n
    return pi

if __name__ == '__main__':
    N = 10**5  # total de iterações
    P = 5      # número de processos
    
    p = Pool(P)
    print('tempo', timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} iterações totais com {P} processos')

Overwriting test.py


In [5]:
! python test.py

3.1364400
3.1448800
3.1400000
3.1467600
3.1419600
3.1392400
3.1425200
3.1396800
3.1478000
3.1451200
tempo 0.2638903
100000 iterações totais com 5 processos


O teste acima levou menos de um segundo em nosso computador.

Agora que sabemos que nosso script funciona, vamos aumentar o número de iterações e comparar dois conjuntos diferentes. Sente-se, isso pode demorar um pouco!

In [6]:
%%writefile test.py
from random import random
from multiprocessing import Pool
import timeit

def find_pi(n):
    """
    Função para estimar o valor de Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # se i cai dentro do círculo
            inside+=1

    pi=4*inside/n
    return pi

if __name__ == '__main__':
    N = 10**5  # total de iteracoes 
    
    P = 1      # numero de processos
    p = Pool(P)
    print('tempo1', timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} total de iteracoes com {P} processos')
    
    P = 5      # número de processos
    p = Pool(P)
    print('tempo2', timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} total de iteracoes com {P} processos')

Overwriting test.py


In [7]:
! python test.py

3.1404400
3.1398400
3.1462800
3.1356800
3.1453600
3.1418800
3.1401600
3.1339200
3.1382800
3.1432800
tempo1 0.43114060000000004
100000 total de iteracoes com 1 processos
3.1325600
3.1442800
3.1397200
3.1431600
3.1405600
3.1406000
3.1432400
3.1483600
3.1430800
3.1414800
tempo2 0.25605100000000003
100000 total de iteracoes com 5 processos


Espero que você tenha visto que, com 5 processos, nosso script correu mais rápido!

In [8]:
%%writefile test2.py
from random import random
from multiprocessing import Pool
import timeit
import sys

N = int(sys.argv[1])  # estes argumentos são passados da linha de comando
P = int(sys.argv[2])

def find_pi(n):
    """
    Função para estimar o valor de Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # se i cai dentro do círculo
            inside+=1

    pi=4*inside/n
    return pi

if __name__ == '__main__':
    
    with Pool(P) as p:
        print('tempo', timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.5f}'), number=10))
    print(f'{N} total de iterações com {P} processos')

Overwriting test2.py


In [9]:
! python test2.py 100000 1

3.13960
3.14240
3.14720
3.14684
3.13920
3.14608
3.15068
3.14232
3.14396
3.14624
tempo 0.4792976
100000 total de iterações com 1 processos


In [10]:
! python test2.py 100000 2

3.13372
3.14072
3.14156
3.14272
3.14308
3.14244
3.13984
3.14748
3.14004
3.13664
tempo 0.29992420000000003
100000 total de iterações com 2 processos


In [11]:
! python test2.py 100000 3

3.13763
3.14095
3.14283
3.13303
3.14747
3.14427
3.14419
3.14043
3.14111
3.13703
tempo 0.26222070000000003
100000 total de iterações com 3 processos


In [34]:
! python test2.py 100000 4

3.14192
3.14860
3.14248
3.14700
3.14964
3.14460
3.14724
3.14040
3.14316
3.14872
tempo 0.2315009
100000 total de iterações com 4 processos


In [36]:
! python test2.py 100000 5

3.14764
3.14632
3.14028
3.14556
3.14912
3.14172
3.15448
3.13688
3.14448
3.14636
tempo 0.22189780000000003
100000 total de iterações com 5 processos


In [38]:
! python test2.py 100000 10

3.13356
3.13776
3.14640
3.14320
3.14704
3.14208
3.13708
3.14024
3.13868
3.14368
tempo 0.2095065
100000 total de iterações com 10 processos


Agora você deve ter um bom entendimento de multithreading e multiprocessing!